In [5]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
import cv2
import numpy as np
import PIL
import scipy

Define paths

In [6]:
base_dir = 'D:/ML projects/Kitty mood/data/images' 

Data augmentation procedure

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # 20% for validation
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 280 images belonging to 6 classes.
Found 68 images belonging to 6 classes.


Load the pre-trained MobileNetV2 model and fine-tune it

In [8]:
base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')
base_model.trainable = False
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(6, activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\Kovi\AppData\Local\Temp\ipykernel_40628\4292523440.py:1: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(150, 150, 3), include_top=False, weights='imagenet')


Train the model

In [9]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator
)

ImportError: This requires the scipy module. You can install it via `pip install scipy`

Save the model

In [ ]:
model.save('models/cat_mood_model.keras')
print("Model saved successfully!")

Load Model

In [ ]:
model = tf.keras.models.load_model('models/cat_mood_model.keras')
print("Model loaded successfully!")

Preprocess image

In [ ]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (150, 150))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

Predict mood

In [ ]:
def predict_mood(image_path):
    img = preprocess_image(image_path)
    prediction = model.predict(img)
    class_names = ['angry', 'calm', 'focused', 'groom', 'playful', 'sleepy']
    return class_names[np.argmax(prediction)]

Test the prediction function

In [ ]:
test_image_path = 'testimage/testimg.jpg'  # Ensure this path is correct and the file exists
mood = predict_mood(test_image_path)
print(f'The cat is {mood}.')